In [ ]:
%load_ext autoreload

In [ ]:
import lightning.pytorch as pl
import torch
import os

In [ ]:
%autoreload 2
from self_supervision.estimator.cellnet import EstimatorAutoEncoder
from self_supervision.tester.reconstruction.test import eval_rec, test_models_on_ood, test_random_model
from self_supervision.paths import DATA_DIR, TRAINING_FOLDER

In [ ]:
STORE_DIR = os.path.join(DATA_DIR, 'merlin_cxg_2023_05_15_sf-log1p')

### Initialize the lightning model

In [ ]:
# init estim class
estim = EstimatorAutoEncoder(STORE_DIR)

In [ ]:
# init datamodule
estim.init_datamodule(batch_size=1024) 

In [ ]:
# init model
hidden_units = [512, 512, 256, 256, 64]
estim.init_model(
    model_type='mlp_ae',
    model_kwargs={
        'learning_rate': 1e-3,
        'weight_decay': 0.1,
        'lr_scheduler': torch.optim.lr_scheduler.StepLR,
        'lr_scheduler_kwargs': {
            'step_size': 2,
            'gamma': 0.9,
            'verbose': True
        },
        'units_encoder': hidden_units,
        'units_decoder': hidden_units[::-1][1:],
    },
)

In [ ]:
estim.trainer = pl.Trainer(logger=[], accelerator='cpu', devices=1)

### Load all models of interest

In [ ]:
"""
model_dirs = [
    ### Supervised Models ###
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun0/default/version_3/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun1/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun2/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun3/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun4/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    ### Only Pretrained Models ###
    # Individual Gene Masking
    TRAINING_FOLDER + '/pretext_models/masking/CN_MLP_50p/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_MLP_50prun1/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_MLP_50prun2/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_MLP_50prun3/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_MLP_50prun4/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    # Gene Program Masking
    TRAINING_FOLDER + '/pretext_models/masking/CN_MLP_gene_program_C8_25p/default/version_2/checkpoints/best_checkpoint_val.ckpt',

    # Gene Program to Gene Program
    TRAINING_FOLDER + '/pretext_models/masking/CN_MLP_single_gene_program/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    
    # Gene Program to Transcription Factor    
    TRAINING_FOLDER + '/pretext_models/masking/CN_MLP_gp_to_tf/default/version_2/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_MLP_gp_to_tfrun1/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_MLP_gp_to_tfrun2/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_MLP_gp_to_tfrun3/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_MLP_gp_to_tfrun4/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    # Boostrap Your Own Latent (BYOL)
    # TRAINING_FOLDER + '/pretext_models/contrastive/MLP_BYOL_Gaussian_0_001_v4/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    # Barlow Twins
    # TRAINING_FOLDER + '/pretext_models/contrastive/MLP_bt_Gaussian_0_001/best_checkpoint_val.ckpt',
    
    ### Self-Supervised Models ###
    # Individual Gene Masking
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_50prun0/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_50prun1/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_50prun2/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_50prun3/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_50prun4/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    # Gene Program Masking
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gene_program_C8_25prun0/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    
    # Gene Program to Gene Program    
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_single_gene_programrun0/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    
    # Gene Program to Transcription Factor
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfrun0/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfrun1/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfrun2/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfrun3/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfrun4/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    # Boostrap Your Own Latent (BYOL)    
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_MLP_BYOL_Gaussian_0_001run0/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    # Barlow Twins    
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_contrastive_MLP_bt_Gaussian_0_01run0/default/version_0/checkpoints/best_checkpoint_val.ckpt',
]
"""

In [ ]:

# Negative Binomial Fitting
model_dirs = [
    ### Supervised Models ###
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_run0NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_run1NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_run2NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_run3NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_run4NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    ### Only Pretrained Models ###
    # Random Mask
    TRAINING_FOLDER + '/pretext_models/masking/CN_NegBin_50prun0/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_NegBin_50prun1/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_NegBin_50prun2/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_NegBin_50prun3/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_NegBin_50prun4/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    
    # Gene Program Mask
    # TO DO, GOT DELETED
    
    # GP to GP
    TRAINING_FOLDER + '/pretext_models/masking/CN_NegBin_single_gene_programrun0/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    
    # GP to TF
    TRAINING_FOLDER + '/pretext_models/masking/CN_NegBin_gp_to_tfrun0/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_NegBin_gp_to_tfrun1/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_NegBin_gp_to_tfrun2/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_NegBin_gp_to_tfrun3/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_NegBin_gp_to_tfrun4/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    
    ### Self-Supervised Models ###
    # Random Mask
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_50prun0NegBin_/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    
    # Gene Program Mask
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gene_program_C8_25prun0NegBin_/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    
    # GP to GP
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_single_gene_programrun0NegBin_/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    
    # GP to TF
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0run0NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0run1NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0run2NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0run3NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0run4NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    # BYOL
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_MLP_BYOL_Gaussian_0_001_v4run0NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    # Barlow Twins
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_contrastive_MLP_bt_Gaussian_0_01run0NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
]

### Evaluate all models

In [ ]:
metrics = eval_rec(estim, model_dirs)

# Fig 3: OOD Reconstruction

**Dissection: Tail of Hippocampus (HiT) - Caudal Hippocampus - CA4-DGC**

- 56,367 cells
- 10x 3' v3
- hippocampal formation
- astrocyte (3761), central nervous system macrophage (1782), endothelial cell (174), ependymal cell (111), ~~fibroblast (86)~~, leukocyte (36), neuron (36588), oligodendrocyte (11875), oligodendrocyte precursor cell (1896), pericyte (39), vascular associated smooth muscle cell (19)

In [ ]:
ood_set = 'tail_of_hippocampus'

In [ ]:
"""
model_dirs = [
    ### Supervised Models ###
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun0/default/version_3/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun1/default/version_0/checkpoints/best_checkpoint_val.ckpt', 
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun2/default/version_0/checkpoints/best_checkpoint_val.ckpt', 
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun3/default/version_0/checkpoints/best_checkpoint_val.ckpt', 
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun4/default/version_0/checkpoints/best_checkpoint_val.ckpt', 
    
    ### Only Pretrained Models ###
    
    # Gene Program to Transcription Factor    
    TRAINING_FOLDER + '/pretext_models/masking/CN_MLP_gp_to_tf/default/version_2/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_MLP_gp_to_tfrun1/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_MLP_gp_to_tfrun2/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_MLP_gp_to_tfrun3/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_MLP_gp_to_tfrun4/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    
    ### Self-Supervised Models ###
    # Gene Program to Transcription Factor
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfrun0/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfrun1/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfrun2/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfrun3/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfrun4/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
]
"""

In [ ]:
test_models_on_ood(estim, ood_set=ood_set, model_dirs=model_dirs)

**All non-neuronal cells**

- 888,263 cells
- 10x 3' v3
- Bergmann glial cell (8041), astrocyte (155025), central nervous system macrophage (91383), ~~choroid plexus epithelial cell (7689)~~, endothelial cell (5165), ependymal cell (5882), ~~fibroblast (9156)~~, oligodendrocyte (494966), oligodendrocyte precursor cell (105734), pericyte (3693), vascular associated smooth muscle cell (1074)

In [ ]:
ood_set = 'non_neuronal'

In [ ]:
test_models_on_ood(estim, ood_set=ood_set, model_dirs=model_dirs)

In [ ]:
ood_set = 'cort_dev'

In [ ]:
test_models_on_ood(estim, ood_set=ood_set, model_dirs=model_dirs)

In [ ]:
ood_set = 'circ_imm'

In [ ]:
test_models_on_ood(estim, ood_set=ood_set, model_dirs=model_dirs)

In [ ]:
ood_set = 'great_apes'

In [ ]:
test_models_on_ood(estim, ood_set=ood_set, model_dirs=model_dirs)

# Fig 4: Individual Atlas

### HLCA

In [ ]:
model_dirs = [
    ### Only Pretrained Models ###
    TRAINING_FOLDER + '/pretext_models/masking/CN_NegBin_gp_to_tfrun0/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_NegBin_gp_to_tfrun1/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_NegBin_gp_to_tfrun2/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_NegBin_gp_to_tfrun3/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_NegBin_gp_to_tfrun4/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    
    
    ### Supervised Models ###
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_new_run0NegBin__HLCA/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_new_run1NegBin__HLCA/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_new_run2NegBin__HLCA/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_new_run3NegBin__HLCA/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_new_run4NegBin__HLCA/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    
    ### Self-Supervised Models ###
    # Gene Program to Transcription Factor
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run0NegBin__HLCA/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run1NegBin__HLCA/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run2NegBin__HLCA/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run3NegBin__HLCA/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run4NegBin__HLCA/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
]

In [ ]:
supervised_subset = 148  # HLCA

In [ ]:
metrics = test_random_model(estim, supervised_subset=supervised_subset)

In [ ]:
metrics = eval_rec(estim, model_dirs, supervised_subset=supervised_subset)

### PBMC

In [ ]:
"""
model_dirs = [
    ### Only Pretrained Models ###
    
    # Gene Program to Transcription Factor    
    TRAINING_FOLDER + '/pretext_models/masking/CN_MLP_gp_to_tf/default/version_2/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_MLP_gp_to_tfrun1/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_MLP_gp_to_tfrun2/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_MLP_gp_to_tfrun3/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_MLP_gp_to_tfrun4/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    
    ### Supervised Models ###
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run0_PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run1_PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run2_PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run3_PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run4_PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    ### Self-Supervised Models ###
    # Gene Program to Transcription Factor
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfnew_run0_PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfnew_run1_PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfnew_run2_PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfnew_run3_PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfnew_run4_PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
]
"""

In [ ]:
model_dirs = [
    ### Only Pretrained Models ###
    TRAINING_FOLDER + '/pretext_models/masking/CN_NegBin_gp_to_tfrun0/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_NegBin_gp_to_tfrun1/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_NegBin_gp_to_tfrun2/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_NegBin_gp_to_tfrun3/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_NegBin_gp_to_tfrun4/default/version_1/checkpoints/best_checkpoint_val.ckpt', 
    
    ### Supervised Models ###
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_new_run0NegBin__PBMC/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_new_run1NegBin__PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_new_run2NegBin__PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_new_run3NegBin__PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_new_run4NegBin__PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    
    ### Self-Supervised Models ###
    # Gene Program to Transcription Factor
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run0NegBin__PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run1NegBin__PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run2NegBin__PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run3NegBin__PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run4NegBin__PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
]

In [ ]:
supervised_subset = 41  # PBMC

In [ ]:
metrics = test_random_model(estim, supervised_subset=supervised_subset)

In [ ]:
metrics = eval_rec(estim, model_dirs, supervised_subset=supervised_subset)

### Tabula Sapiens

In [ ]:
"""
model_dirs = [
    ### Only Pretrained Models ###
    
    # Gene Program to Transcription Factor    
    TRAINING_FOLDER + '/pretext_models/masking/CN_MLP_gp_to_tf/default/version_2/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_MLP_gp_to_tfrun1/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_MLP_gp_to_tfrun2/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_MLP_gp_to_tfrun3/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_MLP_gp_to_tfrun4/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    
    ### Supervised Models ###
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run0_Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run1_Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run2_Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run3_Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run4_Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    ### Self-Supervised Models ###
    # Gene Program to Transcription Factor
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfnew_run0_Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfnew_run1_Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfnew_run2_Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfnew_run3_Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfnew_run4_Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
]
"""

In [ ]:
model_dirs = [
    ### Only Pretrained Models ###
    TRAINING_FOLDER + '/pretext_models/masking/CN_NegBin_gp_to_tfrun0/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_NegBin_gp_to_tfrun1/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_NegBin_gp_to_tfrun2/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_NegBin_gp_to_tfrun3/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/pretext_models/masking/CN_NegBin_gp_to_tfrun4/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    
    
    ### Supervised Models ###
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_new_run0NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_new_run1NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_new_run2NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_new_run3NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_new_run4NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_new_run5NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_new_run6NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_new_run7NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_new_run8NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_No_SSL_CN_new_run9NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    ### Self-Supervised Models ###
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run0NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run1NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run2NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run3NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    TRAINING_FOLDER + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run4NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
]

In [ ]:
supervised_subset = 87  # Tabula Sapiens

In [ ]:
metrics = test_random_model(estim, supervised_subset=supervised_subset)

In [ ]:
metrics = eval_rec(estim, model_dirs, supervised_subset=supervised_subset)